# Flood Duration
Tralala

In [1]:
%run setup.ipynb
data_dir = Path('../data' )
output_dir = Path('../output') 

In [2]:
# Extracting the necessary data
flood_day = SL_daily_max['flood_day']
time = SL_daily_max['time']
station_names = SL_daily_max['station_name'].values

# Convert time to pandas datetime
time = pd.to_datetime(time.values)

# Initialize a dictionary to hold results
flood_events = {}

# Loop through each station
for i, station in enumerate(station_names):
    station_flood_days = flood_day[:, i].values
    station_flood_days = pd.Series(station_flood_days, index=time)
    
    # Group by year
    station_flood_days_by_year = station_flood_days.groupby(station_flood_days.index.year)
    
    # Initialize list to hold all events for this station
    station_events = []
    
    for year, data in station_flood_days_by_year:
        flood_event_durations = []
        current_event_length = 0
        
        for day in data:
            if day:
                current_event_length += 1
            else:
                if current_event_length > 0:
                    flood_event_durations.append(current_event_length)
                current_event_length = 0
        
        # Append the last event if it was ongoing at the end of the year
        if current_event_length > 0:
            flood_event_durations.append(current_event_length)
        
        # Store the events for this year
        station_events.append({
            "year": year,
            "events": flood_event_durations
        })
    
    # Store the results for the station
    flood_events[station] = station_events

NameError: name 'SL_daily_max' is not defined

In [ ]:
# Setting up the plot grid again with larger station names

# Determine unique years in the dataset
years = sorted(set(year_data["year"] for station_events in flood_events.values() for year_data in station_events))

# Setting up the plot grid
num_stations = len(station_names)
# Determine the 5-year intervals for labeling
five_year_ticks = [year for year in years if year % 5 == 0]

# Setting up the plot grid again with individual subplots and station names inside the plots
fig, axes = plt.subplots(nrows=num_stations, ncols=1, figsize=(6, 10), sharex=True)

# Plotting for each station
for ax, station in zip(axes, station_names):
    yearly_durations = {year: [] for year in years}
    
    # Populate the yearly durations
    for year_data in flood_events[station]:
        year = year_data["year"]
        yearly_durations[year].extend(year_data["events"])
    
    # Prepare data for boxplot
    boxplot_data = [yearly_durations[year] for year in years]
    
    # Create the boxplot
    ax.boxplot(
        boxplot_data, 
        positions=range(len(years)),
        patch_artist=True, 
        boxprops=dict(facecolor='lightblue', color='blue', alpha=0.7),
        medianprops=dict(color='red'),
        whiskerprops=dict(color='blue', alpha=0.5),
        capprops=dict(color='blue', alpha=0.5)
    )
    
    # Add station name inside the plot
    ax.text(2, ax.get_ylim()[1] * 0.8, station, fontsize=12, ha='left', color='black')
    ax.set_ylabel('Duration (days)', fontsize=10)
    ax.grid(False)

# Clean up and remove x-axis labels from all but the last subplot
for ax in axes[:-1]:
    ax.set_xticks([])

# Set x-ticks and labels for every 5 years
axes[-1].set_xticks([years.index(year) for year in five_year_ticks])
axes[-1].set_xticklabels(five_year_ticks, ha='right')
axes[-1].set_xlabel('Year', fontsize=12)



In [ ]:
# Loop through each sensor column
for column in flood_hour_df.columns:
    flood_duration_day[column] = []
    
    # Find the indices where flooding starts and ends
    flood_events = df_flood[column].ne(df_flood[column].shift()).cumsum()
    
    # Group by the flood event indices, filtering out False events
    for event_id, group in df_flood.groupby(flood_events):
        if group[column].iloc[0]:  # Only consider True (flood) events
            start_time = group.index.min()
            end_time = group.index.max()
            duration = end_time - start_time 
            # round to nearest hour
            duration = np.round(duration.total_seconds() / 3600) + 1
            flood_durations[column].append((start_time, duration))
            height = flood_data_df.loc[start_time:end_time, column].max() # max height
            flood_heights[column].append((start_time, height))

In [ ]:
# flood_hour = (hourly_data.sea_level_MHHW.values > thresholds[:,None])
flood_hour = (hourly_data.sea_level_MHHW.values > 30)
flood_hour
flood_hour = np.transpose(flood_hour)


In [ ]:
flood_hour_df = pd.DataFrame(flood_hour, index = hourly_data.time.values, columns = hourly_data.record_id.values)

flood_hour_df

In [ ]:
# get sea level data where flood_hour is True

flood_data = hourly_data.sea_level_MHHW.where(flood_hour.T)
flood_data_df = pd.DataFrame(flood_data.T-30, index = hourly_data.time.values, columns = hourly_data.record_id.values)
flood_data_df

In [ ]:
# Find indices of flood events, and the duration of each event for record id = 57
# Initialize a dictionary to store the durations
flood_durations = {}
flood_heights = {}


df_flood = flood_hour_df 

# Loop through each sensor column
for column in flood_hour_df.columns:
    flood_durations[column] = []
    flood_heights[column] = []
    
    # Find the indices where flooding starts and ends
    flood_events = df_flood[column].ne(df_flood[column].shift()).cumsum()
    
    # Group by the flood event indices, filtering out False events
    for event_id, group in df_flood.groupby(flood_events):
        if group[column].iloc[0]:  # Only consider True (flood) events
            start_time = group.index.min()
            end_time = group.index.max()
            duration = end_time - start_time 
            # round to nearest hour
            duration = np.round(duration.total_seconds() / 3600) + 1
            flood_durations[column].append((start_time, duration))
            height = flood_data_df.loc[start_time:end_time, column].max() # max height
            flood_heights[column].append((start_time, height))




In [ ]:
# make dataframe from dictionary
# Assuming flood_durations and hourly_data are already defined

# Initialize an empty list to store DataArrays
data_arrays = []

# Loop through each record_id and create a DataArray
for record_id in hourly_data.record_id.values:
    flood_durations_df = pd.DataFrame(flood_durations[record_id])
    flood_durations_df.columns = ['time', 'duration']

    flood_heights_df = pd.DataFrame(flood_heights[record_id])   
    flood_heights_df.columns = ['time', 'height']
    
    # Merge the two DataFrames on 'time'
    merged_df = pd.merge(flood_durations_df, flood_heights_df, on='time')
    
    # Create a DataArray with both 'duration' and 'height'
    flood_data_da = xr.DataArray(
        merged_df[['duration', 'height']].values,
        dims=['time', 'variable'],
        coords={'time': merged_df.time.values, 'variable': ['duration', 'height'], 'record_id': record_id}
    )
    
    data_arrays.append(flood_data_da)


# Combine all DataArrays into a single Dataset
flood_data_ds = xr.concat(data_arrays, dim='record_id')

# Convert to Dataset
flood_data_ds = flood_data_ds.to_dataset(dim='variable')

flood_data_ds


In [ ]:
# make storm time by shifting time by 4 months, such that May 1st is the start of the storm season
# Convert the time values to pandas DatetimeIndex
time_values = pd.to_datetime(flood_data_ds.time.values)

# Jan 1 should be the equivalent of May 1
shifted_time_values = time_values + pd.DateOffset(months=4)

#remove anything in 1982
shifted_time_values = shifted_time_values[shifted_time_values.year != 1982]


# Assign the shifted time values back to the dataset
flood_data_ds['time'] = shifted_time_values


flood_data_ds

In [ ]:
# scatterplot flood duration vs year
from matplotlib.colors import LinearSegmentedColormap

# make figure size small
plt.figure(figsize=(10,4))

# Create a custom color ramp and exclude the lightest shade
full_palette = sns.color_palette("Blues", n_colors=256)
# adjusted_heatmap_palette = full_palette[50:]  # Skip the lightest 50 shades

# Convert the adjusted palette to a colormap
# custom_palette = LinearSegmentedColormap.from_list("custom_blues", adjusted_heatmap_palette)
custom_palette = sns.color_palette("OrRd", as_cmap=True)


# selected_data['duration'] = selected_data['duration'].astype(float)
# flood_durations_ds['month'] = flood_durations_ds['month'].astype(float)

record_ids = flood_data_ds['record_id'].values
station_names = hourly_data['station_name'].sel(record_id=record_ids).values

for i, record_id in enumerate(record_ids):
    # Select data for the current record_id and drop NaNs
    selected_data = flood_data_ds.sel(record_id=record_id).dropna(dim='time')
    
    # Ensure 'duration' is numeric
    selected_data['duration'] = selected_data['duration'].astype(float)
    
    # Plot each station's data on a different y-coordinate
    scatter = plt.scatter(selected_data['time'], [i + 1] * len(selected_data['time']), s=10 * selected_data['duration'], c=selected_data['height'], cmap=custom_palette)

plt.xlabel('Time')
plt.yticks(range(1, len(record_ids) + 1), hourly_data['station_name'].sel(record_id=record_ids).values)  # Set y-ticks to show station labels
plt.colorbar(scatter, label='Max height \nabove threshold (cm)')

# show the colorbar